In [1]:
import os
import pickle
import scipy.io
import numpy as np
import numpy.linalg


from scfw.scopt import scopt
from scfw.prox_grad import prox_grad
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
data_folder = '../data'

problem_name = 'syn_1000_1200_10_50_2'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [3]:
results_folder = './results/'
results_file=os.path.join(results_folder,'portfolio',problem_name+'.pckl')
results = {problem_name: {}}
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results=pickle.load(f)
        except Exception: # so many things could go wrong, can't be more specific.
            pass 

In [4]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [5]:
N, n = R.shape
Mf = 2
nu = 3

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
f0,dot_product0=pr.portfolio(R,x0)
sigma_f=min(np.linalg.eigvalsh(pr.hess_portfolio(R, x0, dot_product0)))
if sigma_f<0:
    sigma_f=1e-10
terminate_tol = 1e-15

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
    'rho':np.sqrt(n), #parameters for ll00
    'diam_X':np.sqrt(2),
    'sigma_f':sigma_f,                   
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [6]:
func_x = lambda x: pr.portfolio(R,x)
func_beta = lambda x, s, beta, dot_product, dot_product_s:lr.log_reg(R, (1 - beta) * x + beta * s, (dot_product)*(1-beta)+(dot_product_s)*beta,gamma)
func_beta = lambda x, s, beta, dot_product, dot_product_s: pr.portfolio(R,(1 - beta) * x + beta * s,(1 - beta) * dot_product + beta * dot_product_s)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
prox_func = lambda x, L: pr.proj_simplex(x)

## Prox grad

In [13]:
def run_prox_grad(problem_name):
    data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
    R = data['W']
    results = {problem_name: {}}
    results_file = os.path.join(results_folder,'portfolio', problem_name + '.pckl')
    results = {problem_name: {}}
    if os.path.isfile(results_file):
        with open(results_file, "rb") as f:
                results=pickle.load(f)
    N, n = R.shape
    Mf = 2
    nu = 3
    I = np.eye(n)
    Iv = np.ones(n)
    #running parameters
    x0 = np.ones(n) / n
    terminate_tol = 1e-15
    prox_params={
        #parameters for SCOPT
        'iter_prox': 1000,
        'Lest': 'backtracking',#,'estimate', #estimate L
        'bb_type': 2,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
    }
    func_x = lambda x: pr.portfolio(R,x)
    func_beta = lambda x, s, beta, dot_product, dot_product_s:lr.log_reg(R, (1 - beta) * x + beta * s, (dot_product)*(1-beta)+(dot_product_s)*beta,gamma)
    func_beta = lambda x, s, beta, dot_product, dot_product_s: pr.portfolio(R,(1 - beta) * x + beta * s,(1 - beta) * dot_product + beta * dot_product_s)
    grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
    grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
    hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
    hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
    hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
    extra_func = lambda x: R @ x
    linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
    llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
    prox_func = lambda x, L: pr.proj_simplex(x)
    
    x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
            grad_x,
            hess_mult_vec_x,
            prox_func,
            Mf,
            x0,
            prox_params,
            eps=terminate_tol,
            print_every=200)
    results[problem_name]['prox_grad'] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)  

In [14]:
data_list = ['syn_1000_800_10_50', 'syn_1000_1200_10_50', 'syn_1000_1500_10_50',
             'syn_1000_800_10_50_1', 'syn_1000_1200_10_50_1', 'syn_1000_1500_10_50_1',
            'syn_1000_800_10_50_2', 'syn_1000_1200_10_50_2', 'syn_1000_1500_10_50_2',
            'syn_1000_800_10_50_3', 'syn_1000_1200_10_50_3', 'syn_1000_1500_10_50_3']
for problem_name in data_list:
    run_prox_grad(problem_name)

iter =    1, stepsize = 4.306e-02, rdiff = 5.793e-05 , f = 0.0563501
boo
boo
Convergence achieved!
iter =  105, stepsize = 1.000e+00, rdiff = 5.824e-16,value=-4.57141
8.750502109527588
iter =    1, stepsize = 3.926e-02, rdiff = 4.238e-05 , f = 0.0803035
boo
boo
boo
Convergence achieved!
iter =  194, stepsize = 1.000e+00, rdiff = 2.583e-16,value=-5.11902
21.94225239753723
iter =    1, stepsize = 3.647e-02, rdiff = 3.650e-05 , f = 0.0212893
boo
boo
Convergence achieved!
iter =  149, stepsize = 1.000e+00, rdiff = 3.628e-16,value=-4.47509
28.235453844070435
iter =    1, stepsize = 4.840e-02, rdiff = 5.154e-05 , f = 0.021242
boo
boo
Convergence achieved!
iter =   97, stepsize = 1.000e+00, rdiff = 3.174e-16,value=-3.95931
7.4494359493255615
iter =    1, stepsize = 3.806e-02, rdiff = 4.371e-05 , f = -0.0822446
boo
boo
boo
boo
Convergence achieved!
iter =  138, stepsize = 1.000e+00, rdiff = 1.808e-16,value=-4.98384
18.327462434768677
iter =    1, stepsize = 3.647e-02, rdiff = 3.650e-05 , f = 0

## Run FW

In [8]:
run_alpha_policies = ['backtracking','sc','sc_backtracking','sc_hybrid']

for policy in run_alpha_policies:
    print(policy)
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x, 
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    
with open(results_file, 'wb') as f:
        pickle.dump(results, f)

backtracking
********* Algorithm starts *********
iter = 1, stepsize = 1, criterion = 1e-05, upper_bound=-0.05387037986762075, lower_bound=-6.345730260495745, real_Gap=6.291859880628124
iter = 1000, stepsize = 1.3720835384843286e-08, criterion = 4.152105730115517e-10, upper_bound=-5.398711166599411, lower_bound=-5.398711167082826, real_Gap=4.834150857391251e-10
iter = 2000, stepsize = 7.495705148297297e-08, criterion = 2.597575402945684e-10, upper_bound=-5.398711166599411, lower_bound=-5.398711167082826, real_Gap=4.834150857391251e-10
iter = 3000, stepsize = 6.069722025913335e-08, criterion = 5.926412547907496e-11, upper_bound=-5.398711166599411, lower_bound=-5.398711167082826, real_Gap=4.834150857391251e-10
iter = 4000, stepsize = 1.1277560265120544e-07, criterion = 5.926412547907496e-11, upper_bound=-5.398711166599411, lower_bound=-5.398711167082826, real_Gap=4.834150857391251e-10
iter = 5000, stepsize = 3.8686135565360995e-08, criterion = 4.132538947285425e-12, upper_bound=-5.398711

iter = 46000, stepsize = 3.317215318105653e-08, criterion = 1.1014662117704049e-12, upper_bound=-5.3987111666037935, lower_bound=-5.398711166614673, real_Gap=1.0879297462906834e-11
iter = 47000, stepsize = 9.491642854459423e-08, criterion = 1.1014662117704049e-12, upper_bound=-5.3987111666037935, lower_bound=-5.398711166614673, real_Gap=1.0879297462906834e-11
iter = 48000, stepsize = 4.443183314882165e-09, criterion = 1.1014662117704049e-12, upper_bound=-5.3987111666037935, lower_bound=-5.398711166614673, real_Gap=1.0879297462906834e-11
iter = 49000, stepsize = 5.226568495540202e-10, criterion = 1.1014662117704049e-12, upper_bound=-5.3987111666037935, lower_bound=-5.398711166614673, real_Gap=1.0879297462906834e-11
iter = 50000, stepsize = 3.706815648281812e-08, criterion = 1.1014662117704049e-12, upper_bound=-5.3987111666037935, lower_bound=-5.398711166614673, real_Gap=1.0879297462906834e-11
382.33314204216003
sc
********* Algorithm starts *********
iter = 1, stepsize = 0.4933068320535

iter = 45000, stepsize = 3.0637375551521296e-05, criterion = 1e-05, upper_bound=-5.398697348950035, lower_bound=-5.398718693315656, real_Gap=2.1344365620734607e-05
iter = 46000, stepsize = 2.9971118726566584e-05, criterion = 1e-05, upper_bound=-5.398697649456066, lower_bound=-5.398718529634236, real_Gap=2.088017816959109e-05
iter = 47000, stepsize = 2.9333214877890707e-05, criterion = 9.818906977190185e-06, upper_bound=-5.398697937172731, lower_bound=-5.398718372918129, real_Gap=2.0435745398117433e-05
iter = 48000, stepsize = 2.8721903159728437e-05, criterion = 9.614261442599627e-06, upper_bound=-5.398698212895052, lower_bound=-5.39871822273545, real_Gap=2.0009840397960943e-05
iter = 49000, stepsize = 2.8135551698344944e-05, criterion = 9.417971801896833e-06, upper_bound=-5.3986984773588045, lower_bound=-5.398718078684935, real_Gap=1.9601326130214147e-05
iter = 50000, stepsize = 2.7572664856051123e-05, criterion = 9.229538638097677e-06, upper_bound=-5.398698731238358, lower_bound=-5.39

In [9]:
run_alpha_policies = []#["backtracking","standard","line_search","icml"]

for policy in run_alpha_policies:
    print(policy)
    #policy_results = []
    #for i in range(len(points)):
    #    x0 = points[i]
    x, alpha_hist, Gap_hist, Q_hist, time_hist = frank_wolfe(func_x,
                        func_beta,                                       
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,                                       
                        alpha_policy=policy,                                                    
                        eps=terminate_tol,
                        print_every=1000, 
                        debug_info=False)
        
    results[problem_name][policy]={
    'x': x,
    'alpha_hist': alpha_hist,        
    'Q_hist': Q_hist,
    'Gap_hist': Gap_hist,        
    'time_hist': time_hist,}
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
#    results[problem_name][policy] = policy_results

## Run SCOPT

In [15]:
x, alpha_hist, Q_hist, time_hist = scopt(func_x,
        grad_x,
        hess_mult_x,
        hess_mult_vec_x,
        Mf,
        nu,
        prox_func,
        x0,  
        sc_params,                                              
        eps=terminate_tol,                                              
        print_every=1)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
#    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
#    'grad_hist': grad_hist
}

Fista err = 9.813e-06; Subiter =  45; subproblem converged!
iter =    1, stepsize = 4.745e-01, rdiff = 6.787e-01 , f = -0.0538704
Fista err = 9.244e-06; Subiter =  37; subproblem converged!
iter =    2, stepsize = 6.329e-01, rdiff = 3.575e-01 , f = -2.74112
Fista err = 9.955e-06; Subiter = 212; subproblem converged!
iter =    3, stepsize = 8.242e-01, rdiff = 1.318e-01 , f = -4.46202
Fista err = 9.956e-06; Subiter = 154; subproblem converged!
iter =    4, stepsize = 9.641e-01, rdiff = 2.302e-02 , f = -5.23731
Fista err = 2.253e-11; Subiter =   2; subproblem converged!
iter =    5, stepsize = 9.988e-01, rdiff = 7.388e-04 , f = -5.39294
Fista err = 7.434e-06; Subiter =   2; subproblem converged!
iter =    6, stepsize = 9.999e-01, rdiff = 8.368e-05 , f = -5.3987
Fista err = 5.344e-06; Subiter =   2; subproblem converged!
iter =    7, stepsize = 1.000e+00, rdiff = 1.165e-05 , f = -5.39871
Fista err = 2.691e-10; Subiter =   2; subproblem converged!
iter =    8, stepsize = 1.000e+00, rdiff = 